# Chapter 09. 로지스틱 회귀의 기초
## 3. 로지스틱 회귀 구현하기
### 01 - 로지스틱 회귀 구현을 위한 함수

#### 1.1 시그모이드 함수
- 첫 번째로 필요한 함수는 시그모이드 함수(sigmoid function)
    - $h_\theta (x) = g(z) = \frac{1}{1 + e^{-z}}$
    - 넘파이의 np.exp 함수를 사용하여 구현

#### 1.2 가설함수
- 두 번째는 기존 시그모이드 함수를 약간 업데이트한 가설함수(hypothesis function)
    - 시그모이드 함수는 단순히 $z$값을 입력값으로 사용
        - 실제로는 가중치와 피쳐의 선형 결합으로 이루어짐
            - 피쳐 값들은 $x$벡터
            - 가중치 값들은 $\theta$
            - 위 두 변수를 입력하는 함수를 만듦
    - $h_\theta (x) = \frac{1}{1 + e^{-\theta^Tx}}$

#### 1.3 비용함수
- 세 번째는 비용함수(cost function)
    - 생성한 가설함수와 실제값 y 간의 차이가 y가 1일 때, 또는 0일 때마다 연산이 달라지는 것을 알 수 있음
    - 넘파이로 표현할 때는 벡터 형태로만 연산하는 방법, 곱셈 & sum을 이용하는 방법이 있음
    - $J(\theta) = \frac{1}{m} \displaystyle\sum_{i = 1}^m {Cost(h_\theta(x^{(i)}), y^{(i)})}$
    - $= -\frac{1}{m} \displaystyle\sum_{i = 1}^m [y^{(i)}\log h_\theta(x^{(i)}) + ( 1 - y^{(i)}) \log (1 - h_\theta(x^{(i)}))]$

In [1]:
import numpy as np

# 1. sigmoid function
def sigmoid(z) :
    return 1 / (1 + np.exp(z))

# 2. hypothesis function
def hypothesis_function(x, theta) :
    z = np.dot(-x, theta)
    return sigmoid(z)

# 3. cost function
def compute_cost(x, y, theta) :
    m = y.shape[0]
    J = (-1.0 / m) * (y.T.dot(np.log(hypothesis_function(x, theta))) + (1 - y).T.dot(np.log(1 - hypothesis_function(x, theta))))
    return J

#### 1.4 경사하강법 : 가중치 업데이트
- 마지막으로 가중치의 업데이트가 발생하는 경사하강법 코드
    - 선형회귀와 동일하게 for 루프가 업데이트 되면서 $\theta$의 개수, 즉 가중치의 개수만큼 경사도를 구하고 이를 예측값인 가설함수의 결과값과 기존 y값 간의 차이에 곱하는 형태의 연산
    - $\theta_j := \theta_j - \alpha \displaystyle\sum_{i = 1}^m {(h_\theta(x^i) - y^i){x^i}_j}$
        - 이 수식에서 뒷 부분에 있는 ${x^i}_j$는 아래 코드에서 **partial_marginal**변수에 저장되어 있음
    - 다음으로 가설함수와 실제값 간의 차이를 **delta**에 저장
    - 마지막으로 **delta, partial_marginal**을 곱한 후 그 합계를 **grad_i**에 저장
    - 앞의 수식과 동일하게 기존 $\theta_i$의 값에서 계산된 경사도 값만큼 빼주면서 가중치 업데이트 수행

In [2]:
def minimize_gradient(x, y, theta, iterations = 100000, alpha = 0.01) :
    m = y.size
    cost_history = []
    theta_history = []

    for _ in range(iterations) :
        original_theta = theta
        for i in range(theta.size) :
            partial_marginal = x[:, i].reshape(x.shape[0], 1)
            delta = hypothesis_function(x, original_theta) - y
            grad_i = delta.T.dot(partial_marginal)

            theta[i] = theta[i] - (alpha * grad_i)

        if (_ % 100) == 0 :
            theta_history.append(theta)
            cost_history.append(compute_cost(x, y, theta))

    return theta, np.array(cost_history), np.array(theta_history)

### 02 - 사이킷런을 사용하여 학습하기
- 사이킷런에서도 로지스틱 회귀를 사용할 수 있음
    - linear_model 모듈 안에 LogisticRegression 클래스 존재

#### 2.1 데이터셋 준비
- 인터넷 사용자가 뉴비인지 아닌지를 구별하는 와튼대학교의 'uva.txt' 데이터 사용

In [3]:
import pandas as pd
data_url = "http://www-stat.wharton.upenn.edu/~waterman/DataSets/uva.txt"

df = pd.read_table(data_url)
df[:5]

,who,Newbie,Age,Gender,Household Income,Sexual Preference,Country,Education Attainment,Major Occupation,Marital Status,Years on Internet
0,id74364,0,54.0,Male,$50-74,Gay male,Ontario,Some College,Computer,Other,4-6 yr
1,id84505,0,39.0,Female,Over $100,Heterosexual,Sweden,Professional,Other,Other,1-3 yr
2,id84509,1,49.0,Female,$40-49,Heterosexual,Washington,Some College,Management,Other,Under 6 mo
3,id87028,1,22.0,Female,$40-49,Heterosexual,Florida,Some College,Computer,Married,6-12 mo
4,id76087,0,20.0,Male,$30-39,Bisexual,New Jersey,Some College,Education,Single,1-3 yr


#### 2.2 데이터 전처리
- 불러온 데이터에 여러 변수가 존재
- 데이터 전처리를 위해 who, Country, Years on Internet 열 제거
    - 개인의 ID 정보인 who는 분석하는 데 의미가 없음
    - 미국의 주를 나타내는 Country는 값이 너무 많기 때문에 이 분석에서는 사용 X
    - Years on Internet은 인터넷을 얼마나 오랫동한 사용했는지에 대한 정보
        - Y 값에 해당하는 Newbie 열과 같은 의미를 가지므로 제거

- 아래와 같이 코드를 사용하여 필요 없는 데이터를 드롭(drop)

In [4]:
df.pop('who')
df.pop('Country')
df.pop('Years on Internet')

df.dtypes

Newbie                    int64
Age                     float64
Gender                   object
Household Income         object
Sexual Preference        object
Education Attainment     object
Major Occupation         object
Marital Status           object
dtype: object

- 다음으로 category 타입의 데이터들에 대해 원핫인코딩 형태로 바꾸기 위해 데이터의 타입을 변환
    - 데이터 변환을 위해 가장 쉬운 방법 중 하나는 astype 함수를 사용하여 변환하는 것

- category 타입에 해당하는 열들의 이름을 정리하기 위해 아래 코드 수행 후 원핫인코딩

In [5]:
category_cols = ["Gender", "Household Income", "Sexual Preference", "Education Attainment", "Major Occupation", "Marital Status"]

for col in category_cols :
    df[col] = df[col].astype('category')

df.dtypes

Newbie                     int64
Age                      float64
Gender                  category
Household Income        category
Sexual Preference       category
Education Attainment    category
Major Occupation        category
Marital Status          category
dtype: object

In [6]:
df_onehot = pd.get_dummies(df)
df_onehot.shape

(19583, 38)

- 다음으로 데이터의 결측값 확인
    - isnull 함수를 사용하면 각 열별 데이터 중 isnull 함수 결과의 합을 구해 보여줌
        - 그 결과 Age 열에만 결측값이 존재하는 것을 확인 가능

In [7]:
df_onehot.isnull().sum()

Newbie                                 0
Age                                  561
Gender_Female                          0
Gender_Male                            0
Household Income_$10-19                0
Household Income_$20-29                0
Household Income_$30-39                0
Household Income_$40-49                0
Household Income_$50-74                0
Household Income_$75-99                0
Household Income_Over $100             0
Household Income_Under $10             0
Sexual Preference_Bisexual             0
Sexual Preference_Gay male             0
Sexual Preference_Heterosexual         0
Sexual Preference_Lesbian              0
Sexual Preference_Transgender          0
Sexual Preference_na                   0
Education Attainment_College           0
Education Attainment_Doctoral          0
Education Attainment_Grammar           0
Education Attainment_High School       0
Education Attainment_Masters           0
Education Attainment_Other             0
Education Attain

- 결측값을 채우는 기법 중 가장 간단한 방법은 loc 함수를 사용하여 Age 열의 평균값으로 채우는 것

In [8]:
df_onehot.loc[pd.isnull(df_onehot['Age']), "Age"] = df_onehot['Age'].mean()

#### 2.3 데이터 분리
- 데이터를 x, y로 나눈 후 이를 다시 테스트와 훈련 형태로 분류해야 함
    - 이를 위해 먼저 전체 코드 중 Newbie 열은 y_data에 할당하고 나머지 데이터는 x_data에 할당하는 코드 작성

In [9]:
x_data = df_onehot.iloc[:, 1:].values
y_data = df_onehot.iloc[:, 0].values.reshape(-1, 1)
y_data.shape, x_data.shape

((19583, 1), (19583, 37))

- 다음으로 x_data에 대해 MinMaxScalar 함수를 사용해 전체 데이터에 대한 스케일링 실시

In [13]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_data = min_max_scaler.fit_transform(x_data)

- train-test split 함수를 적용해 생성된 데이터를 학습 데이터셋과 테스트 데이터셋으로 분리

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = 0.33, random_state = 42)

X_train.shape, X_test.shape

((13120, 37), (6463, 37))

- 최종적으로 LogisticRegression 클래스를 사용하여 학습된 모델 생성
    - 생성된 모델은 간단히 절편만 생성하는 fit_intercept 매개변수만 설정했지만, 생각보다 많은 매개변수들을 제공
        - 대표적으로 regularization을 penalty라는 이름의 매개변수로 설정 가능
        - 또한 penalty의 강도를 조절하는 하이퍼 매개변수 C 값을 설정 가능
        - max_iter, tol은 기본적으로 경사하강법이 가지는 매개변수

In [15]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept = True)
logreg.fit(X_train, y_train.flatten())

LogisticRegression()

In [16]:
LogisticRegression(C = 1.0, class_weight= None, dual= False,
                fit_intercept= True, intercept_scaling= 1, l1_ratio= None,
                max_iter= 100, multi_class= 'warn', n_jobs= None, penalty= 'l2',
                random_state= None, solver= 'warn', tol= 0.0001, verbose= 0,
                warm_start= False)

LogisticRegression(multi_class='warn', solver='warn')

#### 2.4 값 예측하기와 성능 측정하기
- 생성된 모델을 사용하여 실제값을 예측하기 위해 predict 함수 사용

- 아래 코드를 수행시키면 X_test에 존재하는 값 5개까지 모두 0으로 예측됨

In [17]:
logreg.predict(X_test[:5])

array([0, 0, 0, 0, 0], dtype=int64)

- 로지스틱 회귀의 가설함수는 확률을 예측함
    - 바로 위의 코드처럼 값도 예측할 수 있지만 각 값들의 예측값에 대한 확률도 구할 수 있음
        - 사이킷런에서 predict_proba 함수 제공
            - 이 함수에선 각각 0일 때의 확률과 1일 때의 확률을 n $\times$ 2의 행렬 현태로 나타냄
            - 세세하게 각 클래스의 확률을 보고싶을 때 사용

In [18]:
logreg.predict_proba(X_test[:5])

array([[0.56843258, 0.43156742],
       [0.91112572, 0.08887428],
       [0.79481085, 0.20518915],
       [0.85841562, 0.14158438],
       [0.62764603, 0.37235397]])

- 코드는 다양하게 구성할 수 있지만, 기본적으로 y_true 변수에는 실제값인 y_test 값을 복사하고, y_pred에는 모델로 예측된 값들을 할당
    - 그 후 성능을 측정하는 함수를 사용하면 기대하는 결과값이 출력됨

- 실제 성능을 측정하는 코드는 아래와 같음

In [19]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_true = y_test.copy()
y_pred = logreg.predict(X_test)

confusion_matrix(y_true, y_pred)

array([[4487,  275],
       [1350,  351]], dtype=int64)

In [20]:
accuracy_score(y_true, y_pred)

0.7485687761101656